## GEE Asset Exporter

This script uses the geemap package to export monthly Landsat-derived DSWE products (DSWE mask, QC mask, and RGB of composite) and monthly Sentinel-2 based DSWE products. The code is directed to a GEE asset folder, a study area boundary for clipping, and a local output folder. The code iterates through the assets and exports as .tifs, tiling when needed.

Author: James (Huck) Rees, PhD Student, UC Santa Barbara Geography

Date: October 1st, 2025

## Import packages

In [4]:
import ee
import os
import re
import math
import glob
import rasterio
import geopandas as gpd
from rasterio.merge import merge
from rasterio.mask import mask
import geemap
import tempfile
import shutil

ee.Initialize()

## Export monthly Landsat-derived DSWE products

In [5]:
def get_geometry_area_km2(geom):
    return geom.area().divide(1e6).getInfo()

def tile_geometry(geometry, tile_size_km2):
    area_km2 = get_geometry_area_km2(geometry)
    if area_km2 <= tile_size_km2:
        return [geometry]
    num_tiles = math.ceil(area_km2 / tile_size_km2)
    tile_count = math.ceil(math.sqrt(num_tiles))
    bounds = geometry.bounds().coordinates().getInfo()[0]
    min_lon, min_lat = bounds[0]
    max_lon, max_lat = bounds[2]
    lon_step = (max_lon - min_lon) / tile_count
    lat_step = (max_lat - min_lat) / tile_count
    tiles = []
    for i in range(tile_count):
        for j in range(tile_count):
            tile = ee.Geometry.Rectangle([
                [min_lon + i * lon_step, min_lat + j * lat_step],
                [min_lon + (i + 1) * lon_step, min_lat + (j + 1) * lat_step]
            ])
            tiles.append(tile)
    return tiles

def clip_raster_to_shape(raster_path, shapefile_path):
    gdf = gpd.read_file(shapefile_path)

    with rasterio.open(raster_path) as src:
        if gdf.crs != src.crs:
            gdf = gdf.to_crs(src.crs)

        geoms = [geom.__geo_interface__ for geom in gdf.geometry]
        clipped_image, clipped_transform = mask(src, geoms, crop=True)
        clipped_meta = src.meta.copy()
        clipped_meta.update({
            "height": clipped_image.shape[1],
            "width": clipped_image.shape[2],
            "transform": clipped_transform,
            "nodata": 255,
            "compress": "LZW"
        })

    # Save to temp file first
    with tempfile.NamedTemporaryFile(delete=False, suffix=".tif") as tmpfile:
        temp_path = tmpfile.name

    with rasterio.open(temp_path, "w", **clipped_meta) as dest:
        dest.write(clipped_image)

    # Replace original
    shutil.move(temp_path, raster_path)
    print(f"✅ Clipped raster saved (overwritten): {raster_path}")
    
def stitch_tiles(output_folder, basename, shapefile_path):
    tile_paths = glob.glob(os.path.join(output_folder, f"{basename}_tile_*.tif"))
    if not tile_paths:
        print("No tiles found to stitch.")
        return
    if len(tile_paths) == 1:
        os.rename(tile_paths[0], os.path.join(output_folder, f"{basename}.tif"))
    else:
        srcs = [rasterio.open(p) for p in tile_paths]
        mosaic, transform = merge(srcs)
        for src in srcs:
            src.close()
        mosaic[mosaic == -1] = 0
        meta = srcs[0].meta.copy()
        meta.update({
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": transform,
            "compress": "LZW",
            "nodata": 255,
            "dtype": "uint8"
        })
        out_path = os.path.join(output_folder, f"{basename}.tif")
        with rasterio.open(out_path, "w", **meta) as dest:
            dest.write(mosaic)
        for path in tile_paths:
            os.remove(path)

    # Clip the stitched raster
    clip_raster_to_shape(
        os.path.join(output_folder, f"{basename}.tif"),
        shapefile_path
    )

def export_dswe_assets_tiled(asset_root_path, local_output_folder, shapefile_path, start_year, end_year, tile_size_km2=3000):
    os.makedirs(local_output_folder, exist_ok=True)
    asset_list = ee.data.listAssets({'parent': asset_root_path})['assets']

    for asset in asset_list:
        asset_id = asset['name']
        basename = asset_id.split('/')[-1]
        match = re.match(r'DSWE_(\d{4})_(\d{2})', basename)
        if not match:
            continue
        year = int(match.group(1))
        if not (start_year <= year <= end_year):
            continue

        print(f"🚀 Processing {basename}")
        image = ee.Image(asset_id).select('dswe').unmask(-1).int8()
        region = image.geometry()
        output_path = os.path.join(local_output_folder, f"{basename}.tif")

        # Try full export
        try:
            geemap.ee_export_image(
                image,
                filename=output_path,
                scale=30,
                region=region.getInfo()['coordinates'],
                file_per_band=False,
                crs='EPSG:4326'
            )
        except Exception as e:
            print(f"⚠️ Failed full export for {basename}: {e}")

        if not os.path.exists(output_path):
            print(f"🔁 Falling back to tiling for {basename}")
            tiles = tile_geometry(region, tile_size_km2)
            for i, tile in enumerate(tiles):
                tile_path = os.path.join(local_output_folder, f"{basename}_tile_{i}.tif")
                try:
                    geemap.ee_export_image(
                        image.clip(tile),
                        filename=tile_path,
                        scale=30,
                        region=tile.getInfo()['coordinates'],
                        file_per_band=False,
                        crs='EPSG:4326'
                    )
                except Exception as te:
                    print(f"❌ Tile {i} failed: {te}")
            stitch_tiles(local_output_folder, basename, shapefile_path)
        else:
            print(f"✅ Successfully exported {basename} (no tiling needed)")
            clip_raster_to_shape(output_path, shapefile_path)

In [6]:
export_dswe_assets_tiled(
    asset_root_path="projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products",
    local_output_folder=r"C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting",
    shapefile_path=r"C:\Users\huckr\Desktop\UCSB\Okavango\Data\StudyAreas\Delta_UCB\Delta_UCB_WGS84.shp",
    start_year=2023,
    end_year=2023,
    tile_size_km2=10000
)


🚀 Processing DSWE_2023_01
Generating URL ...
An error occurred while downloading.
Total request size (118405416 bytes) must be less than or equal to 50331648 bytes.
🔁 Falling back to tiling for DSWE_2023_01
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_01_tile_0.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_01_tile_1.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_01_tile_2.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_01_tile_3.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_01_tile_4.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavan

KeyboardInterrupt: 

## Export monthly QC masks

In [22]:
def export_qc_assets_tiled(asset_root_path, local_output_folder, shapefile_path, start_year, end_year, tile_size_km2=3000):
    os.makedirs(local_output_folder, exist_ok=True)
    asset_list = ee.data.listAssets({'parent': asset_root_path})['assets']

    for asset in asset_list:
        asset_id = asset['name']
        basename = asset_id.split('/')[-1]  # QC_YYYY_MM
        match = re.match(r'QC_(\d{4})_(\d{2})', basename)
        if not match:
            continue
        year = int(match.group(1))
        if not (start_year <= year <= end_year):
            continue

        print(f"🧪 Processing {basename}")
        image = ee.Image(asset_id).select('expansion_mask').unmask(-1).int8()
        region = image.geometry()
        output_path = os.path.join(local_output_folder, f"{basename}.tif")

        try:
            geemap.ee_export_image(
                image,
                filename=output_path,
                scale=30,
                region=region.getInfo()['coordinates'],
                file_per_band=False,
                crs='EPSG:4326'
            )
        except Exception as e:
            print(f"⚠️ Failed full export for {basename}: {e}")

        if not os.path.exists(output_path):
            print(f"🔁 Falling back to tiling for {basename}")
            tiles = tile_geometry(region, tile_size_km2)
            for i, tile in enumerate(tiles):
                tile_path = os.path.join(local_output_folder, f"{basename}_tile_{i}.tif")
                try:
                    geemap.ee_export_image(
                        image.clip(tile),
                        filename=tile_path,
                        scale=30,
                        region=tile.getInfo()['coordinates'],
                        file_per_band=False,
                        crs='EPSG:4326'
                    )
                except Exception as te:
                    print(f"❌ Tile {i} failed: {te}")
            stitch_tiles(local_output_folder, basename, shapefile_path)
        else:
            print(f"✅ Successfully exported {basename} (no tiling needed)")
            clip_raster_to_shape(output_path, shapefile_path)


In [24]:
export_qc_assets_tiled(
    asset_root_path='projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/QC_Masks',
    local_output_folder=r'C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting',
    shapefile_path=r"C:\Users\huckr\Desktop\UCSB\Okavango\Data\StudyAreas\Delta_UCB\Delta_UCB_WGS84.shp",
    start_year=1991,
    end_year=1991,
    tile_size_km2=10000  # You can reduce this if you see export failures
)


🧪 Processing QC_1991_01
Generating URL ...
An error occurred while downloading.
Total request size (118405416 bytes) must be less than or equal to 50331648 bytes.
🔁 Falling back to tiling for QC_1991_01
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\QC_1991_01_tile_0.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\QC_1991_01_tile_1.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\QC_1991_01_tile_2.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\QC_1991_01_tile_3.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\QC_1991_01_tile_4.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\T

Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\QC_1991_03_tile_6.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\QC_1991_03_tile_7.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\QC_1991_03_tile_8.tif
✅ Clipped raster saved (overwritten): C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\QC_1991_03.tif
🧪 Processing QC_1991_04
Generating URL ...
An error occurred while downloading.
Total request size (118405416 bytes) must be less than or equal to 50331648 bytes.
🔁 Falling back to tiling for QC_1991_04
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\QC_1991_04_tile_0.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\QC_1991_04_tile_1.tif
Gener

KeyboardInterrupt: 

## Export monthly Landsat RGB composites

In [10]:
def export_rgb_assets_tiled(asset_root_path, local_output_folder, shapefile_path, start_year, end_year, tile_size_km2=3000):
    os.makedirs(local_output_folder, exist_ok=True)
    asset_list = ee.data.listAssets({'parent': asset_root_path})['assets']

    for asset in asset_list:
        asset_id = asset['name']
        basename = asset_id.split('/')[-1]
        match = re.match(r'Composite_(\d{4})_(\d{2})', basename)
        if not match:
            continue
        year = int(match.group(1))
        if not (start_year <= year <= end_year):
            continue

        print(f"🎨 Processing {basename}")
        image = ee.Image(asset_id).select(['Red', 'Green', 'Blue'])  # Order: RGB
        region = image.geometry()
        output_path = os.path.join(local_output_folder, f"{basename}.tif")

        # Apply scaling to preserve float precision (optional)
        image = image.multiply(10000).toUint16()

        try:
            geemap.ee_export_image(
                image,
                filename=output_path,
                scale=30,
                region=region.getInfo()['coordinates'],
                file_per_band=False,
                crs='EPSG:4326'
            )
        except Exception as e:
            print(f"⚠️ Failed full export for {basename}: {e}")

        if not os.path.exists(output_path):
            print(f"🔁 Falling back to tiling for {basename}")
            tiles = tile_geometry(region, tile_size_km2)
            for i, tile in enumerate(tiles):
                tile_path = os.path.join(local_output_folder, f"{basename}_tile_{i}.tif")
                try:
                    geemap.ee_export_image(
                        image.clip(tile),
                        filename=tile_path,
                        scale=30,
                        region=tile.getInfo()['coordinates'],
                        file_per_band=False,
                        crs='EPSG:4326'
                    )
                except Exception as te:
                    print(f"❌ Tile {i} failed: {te}")
            stitch_tiles_rgb(local_output_folder, basename, shapefile_path)
        else:
            print(f"✅ Successfully exported {basename} (no tiling needed)")
            clip_raster_to_shape(output_path, shapefile_path)

def stitch_tiles_rgb(output_folder, basename, shapefile_path):
    tile_paths = glob.glob(os.path.join(output_folder, f"{basename}_tile_*.tif"))
    if not tile_paths:
        print("No tiles found to stitch.")
        return
    if len(tile_paths) == 1:
        os.rename(tile_paths[0], os.path.join(output_folder, f"{basename}.tif"))
    else:
        srcs = [rasterio.open(p) for p in tile_paths]
        mosaic, transform = merge(srcs)
        for src in srcs:
            src.close()

        meta = srcs[0].meta.copy()
        meta.update({
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": transform,
            "compress": "LZW",
            "nodata": 0,
            "dtype": "uint16",
            "count": 3  # RGB bands
        })

        out_path = os.path.join(output_folder, f"{basename}.tif")
        with rasterio.open(out_path, "w", **meta) as dest:
            dest.write(mosaic)

        for path in tile_paths:
            os.remove(path)

    clip_raster_to_shape(os.path.join(output_folder, f"{basename}.tif"), shapefile_path)


In [17]:
export_rgb_assets_tiled(
    asset_root_path="projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/Source_LS_Composites",
    local_output_folder=r'C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting',
    shapefile_path=r"C:\Users\huckr\Desktop\UCSB\Okavango\Data\StudyAreas\Delta_UCB\Delta_UCB_WGS84.shp",
    start_year=2023,
    end_year=2023,
    tile_size_km2=5000
)


🎨 Processing Composite_2023_01
Generating URL ...
An error occurred while downloading.
Total request size (532824372 bytes) must be less than or equal to 50331648 bytes.
🔁 Falling back to tiling for Composite_2023_01
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_01_tile_0.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_01_tile_1.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_01_tile_2.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_01_tile_3.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_01_tile_4.tif
Generating URL ...
Please wait ...
Data downloaded to

Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_02_tile_8.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_02_tile_9.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_02_tile_10.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_02_tile_11.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_02_tile_12.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_02_tile_13.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_02_tile_14.tif
Generati

Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_04_tile_0.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_04_tile_1.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_04_tile_2.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_04_tile_3.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_04_tile_4.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_04_tile_5.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\Composite_2023_04_tile_6.tif
Generating UR

KeyboardInterrupt: 

## Export Sentinel-2 based DSWE products

In [26]:
def export_dswe_s2_assets_tiled(asset_root_path, local_output_folder, shapefile_path, start_year, end_year, tile_size_km2=1000):
    os.makedirs(local_output_folder, exist_ok=True)
    asset_list = ee.data.listAssets({'parent': asset_root_path})['assets']

    for asset in asset_list:
        asset_id = asset['name']
        basename = asset_id.split('/')[-1]  # DSWE_YYYY_MM
        match = re.match(r'DSWE_(\d{4})_(\d{2})', basename)
        if not match:
            continue
        year = int(match.group(1))
        if not (start_year <= year <= end_year):
            continue

        print(f"🛰️ Processing {basename}")
        image = ee.Image(asset_id).select('DSWE').unmask(-1).int8()
        region = image.geometry()
        output_path = os.path.join(local_output_folder, f"{basename}.tif")

        try:
            geemap.ee_export_image(
                image,
                filename=output_path,
                scale=10,
                region=region.getInfo()['coordinates'],
                file_per_band=False,
                crs='EPSG:4326'
            )
        except Exception as e:
            print(f"⚠️ Failed full export for {basename}: {e}")

        if not os.path.exists(output_path):
            print(f"🔁 Falling back to tiling for {basename}")
            tiles = tile_geometry(region, tile_size_km2)
            for i, tile in enumerate(tiles):
                tile_path = os.path.join(local_output_folder, f"{basename}_tile_{i}.tif")
                try:
                    geemap.ee_export_image(
                        image.clip(tile),
                        filename=tile_path,
                        scale=10,
                        region=tile.getInfo()['coordinates'],
                        file_per_band=False,
                        crs='EPSG:4326'
                    )
                except Exception as te:
                    print(f"❌ Tile {i} failed: {te}")
            stitch_tiles(local_output_folder, basename, shapefile_path)
        else:
            print(f"✅ Successfully exported {basename} (no tiling needed)")
            clip_raster_to_shape(output_path, shapefile_path)


In [29]:
export_dswe_s2_assets_tiled(
    asset_root_path='projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m',
    local_output_folder=r'C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting',
    shapefile_path=r"C:\Users\huckr\Desktop\UCSB\Okavango\Data\StudyAreas\Delta_UCB\Delta_UCB_WGS84.shp",
    start_year=2023,
    end_year=2023,
    tile_size_km2=3000
)

🛰️ Processing DSWE_2023_01
Generating URL ...
An error occurred while downloading.
Total request size (1065143040 bytes) must be less than or equal to 50331648 bytes.
🔁 Falling back to tiling for DSWE_2023_01
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_01_tile_0.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_01_tile_1.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_01_tile_2.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_01_tile_3.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_01_tile_4.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okav

Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_02_tile_0.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_02_tile_1.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_02_tile_2.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_02_tile_3.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_02_tile_4.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_02_tile_5.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_02_tile_6.tif
Generating UR

Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_03_tile_0.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_03_tile_1.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_03_tile_2.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_03_tile_3.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_03_tile_4.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_03_tile_5.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\DSWE_2023_03_tile_6.tif
Generating URL ...
Please wait ...
Data download

KeyboardInterrupt: 